### 06.03. Setup functions and indexes

In [1]:
#Install prerequisite packages
!pip install llama-index -q
!pip install llama-index-llms-groq -q
!pip install llama-index-embeddings-huggingface -q

In [2]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import Settings
import os
import nest_asyncio
from google.colab import userdata

nest_asyncio.apply()

#Setup the LLM
Settings.llm=Groq(model="llama-3.3-70b-versatile", api_key=userdata.get('GROQ_API_KEY'))

#Setup the embedding model RAG
Settings.embed_model= HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


In [3]:
from typing import List
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

#-------------------------------------------------------------
# Tool 1 : Function that returns the list of items in an order
#-------------------------------------------------------------
def get_order_items(order_id: int) -> List[str] :
    """Given an order Id, this function returns the
    list of items purchased for that order"""

    order_items = {
            1001: ["Laptop","Mouse"],
            1002: ["Keyboard","HDMI Cable"],
            1003: ["Laptop","Keyboard"]
        }
    if order_id in order_items.keys():
        return order_items[order_id]
    else:
        return []

#-------------------------------------------------------------
# Tool 2 : Function that returns the delivery date for an order
#-------------------------------------------------------------
def get_delivery_date(order_id: int) -> str:
    """Given an order Id, this function returns the
    delivery date for that order"""

    delivery_dates = {
            1001: "10-Jun",
            1002: "12-Jun",
            1003: "08-Jun"
    }
    if order_id in delivery_dates.keys():
        return delivery_dates[order_id]
    else:
        return []

#----------------------------------------------------------------
# Tool 3 : Function that returns maximum return days for an item
#----------------------------------------------------------------
def get_item_return_days(item: str) -> int :
    """Given an Item, this function returns the return support
    for that order. The return support is in number of days"""

    item_returns = {
            "Laptop"     : 30,
            "Mouse"      : 15,
            "Keyboard"   : 15,
            "HDMI Cable" : 5
    }
    if item in item_returns.keys():
        return item_returns[item]
    else:
        #Default
        return 45

#-------------------------------------------------------------
# Tool 4 : Vector DB that contains customer support contacts
#-------------------------------------------------------------
#Setup vector index for return policies
support_docs=SimpleDirectoryReader(input_files=["Customer Service.pdf"]).load_data()

splitter=SentenceSplitter(chunk_size=1024)
support_nodes=splitter.get_nodes_from_documents(support_docs)
support_index=VectorStoreIndex(support_nodes)
support_query_engine = support_index.as_query_engine()


### 06.04. Setup the Customer Service AI Agent

In [4]:
from llama_index.core.tools import FunctionTool

#Create tools for the 3 functions and 1 index
order_item_tool = FunctionTool.from_defaults(fn=get_order_items)
delivery_date_tool = FunctionTool.from_defaults(fn=get_delivery_date)
return_policy_tool = FunctionTool.from_defaults(fn=get_item_return_days)

support_tool = QueryEngineTool.from_defaults(
    query_engine=support_query_engine,
    description=(
        "Customer support policies and contact information"
    ),
)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI

#Setup the Agent worker in LlamaIndex with all the Tools
#This is the tool executor process
agent_worker = FunctionCallingAgentWorker.from_tools(
    [order_item_tool,
     delivery_date_tool,
     return_policy_tool,
     support_tool
    ],
    llm=Settings.llm,
    verbose=True
)
#Create an Agent Orchestrator with LlamaIndex
agent = AgentRunner(agent_worker)


### 06.05. Using the customer service Agent

In [6]:
#Get return policy for an order
response = agent.query(
    "What is the return policy for order number 1001"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1001
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1001}
=== Function Output ===
['Laptop', 'Mouse']
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "return policy for order number 1001"}
=== Function Output ===
There is no information available regarding the return policy for order number 1001. The available information only includes customer service hours and contact details.
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Laptop"}
=== Function Output ===
30
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Mouse"}
=== Function Output ===
15
=== LLM Response ===
The return policy for order number 1001 is as follows:
- Laptop: 30 days
- Mouse: 15 days

 Final output : 
 The return policy for order number 1001 is as follows:
- Laptop: 30 days
- Mouse: 15 days


In [7]:
# Three part question
response = agent.query(
    "When is the delivery date and items shipped for order 1003 and how can I contact customer support?"
)

print("\n Final output : \n", response)

Added user message to memory: When is the delivery date and items shipped for order 1003 and how can I contact customer support?
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1003}
=== Function Output ===
08-Jun
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1003}
=== Function Output ===
['Laptop', 'Keyboard']
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "customer support contact information"}
=== Function Output ===
The customer support contact information is 1-987-654-3210 or support@company.com.
=== LLM Response ===
The delivery date for order 1003 is June 8th, and the items shipped are a laptop and a keyboard. For customer support, you can contact 1-987-654-3210 or support@company.com.

 Final output : 
 The delivery date for order 1003 is June 8th, and the items shipped are a laptop and a keyboard. For customer support, you can contact 1-987-654-3210 or support@company.c

In [8]:
#Question about an invalid order number
response = agent.query(
    "What is the return policy for order number 1004"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1004
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===
[]
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "return policy for order number 1004"}
=== Function Output ===
There is no information available regarding the return policy for order number 1004. The available information only includes customer service contact details and operating hours.
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===
[]
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "return policy"}
=== Function Output ===
There is no information available about the return policy.
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "item from order 1004"}
=== Function Output ===
45
=== LLM Response ===
The return policy for order 